In [2]:
# import necessary libraries

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

# Load the data set
movie = pd.read_csv('movie_detail.csv')
movie.head(1)

,imdbId,title,releaseYear,releaseDate,genre,writers,actors,directors,sequel,hitFlop
0,tt0118578,Albela,2001,20 Apr 2001,Romance,Honey Irani (screenplay) | Honey Irani (story)...,Govinda | Aishwarya Rai Bachchan | Jackie Shro...,Deepak Sareen,0.0,2


# Data Preprocessing

In [3]:
# drop unwanted columns : 

movie.drop(columns =['releaseYear','releaseDate','sequel'],inplace = True) 
movie.head(1)

,imdbId,title,genre,writers,actors,directors,hitFlop
0,tt0118578,Albela,Romance,Honey Irani (screenplay) | Honey Irani (story)...,Govinda | Aishwarya Rai Bachchan | Jackie Shro...,Deepak Sareen,2


In [4]:
movie.isnull().sum() 
movie.dropna(inplace = True) # drop null vlues : 

In [5]:
# chek for the null values  : 

movie.isnull().sum()

imdbId       0
title        0
genre        0
writers      0
actors       0
directors    0
hitFlop      0
dtype: int64

In [6]:
# sentence = 'Honey Irani (screenplay) | Honey Irani (story) | Javed Siddiqui (dialogue)'

def joiner(obj):
    l=[] 
    for i in obj.split('|'): #split the string at 3 part at "|"
        l.append(i.split('(')[0].strip())# split at "(" and fetch first part and strip with 0 tab 
        
    return l 


In [7]:
movie['writers'] = movie['writers'].apply(joiner)
movie.head(2)

,imdbId,title,genre,writers,actors,directors,hitFlop
0,tt0118578,Albela,Romance,"[Honey Irani, Honey Irani, Javed Siddiqui]",Govinda | Aishwarya Rai Bachchan | Jackie Shro...,Deepak Sareen,2
1,tt0169102,Lagaan: Once Upon a Time in India,Adventure | Drama | Musical,"[Ashutosh Gowariker, Ashutosh Gowariker, Kumar...",Aamir Khan | Gracy Singh | Rachel Shelley | Pa...,Ashutosh Gowariker,6


In [8]:
def actor_joiner(obj):  
    l=[] 
    for i in obj.split('|'):
        l.append(i.strip())
    return l 

In [9]:
movie['actors'] = movie['actors'].apply(actor_joiner)
movie.head(3)

,imdbId,title,genre,writers,actors,directors,hitFlop
0,tt0118578,Albela,Romance,"[Honey Irani, Honey Irani, Javed Siddiqui]","[Govinda, Aishwarya Rai Bachchan, Jackie Shrof...",Deepak Sareen,2
1,tt0169102,Lagaan: Once Upon a Time in India,Adventure | Drama | Musical,"[Ashutosh Gowariker, Ashutosh Gowariker, Kumar...","[Aamir Khan, Gracy Singh, Rachel Shelley, Paul...",Ashutosh Gowariker,6
3,tt0222024,Hum Tumhare Hain Sanam,Drama | Romance,"[K.S. Adiyaman, Arun Kumar, Ashok Mehta, Reema...","[Shah Rukh Khan, Madhuri Dixit, Salman Khan, A...",K.S. Adiyaman,4


In [10]:
movie.drop(columns =['hitFlop'],inplace = True)

In [11]:
movie['genre'] = movie['genre'].apply(actor_joiner)
movie.head(2)

,imdbId,title,genre,writers,actors,directors
0,tt0118578,Albela,[Romance],"[Honey Irani, Honey Irani, Javed Siddiqui]","[Govinda, Aishwarya Rai Bachchan, Jackie Shrof...",Deepak Sareen
1,tt0169102,Lagaan: Once Upon a Time in India,"[Adventure, Drama, Musical]","[Ashutosh Gowariker, Ashutosh Gowariker, Kumar...","[Aamir Khan, Gracy Singh, Rachel Shelley, Paul...",Ashutosh Gowariker


In [12]:
movie['directors'] = movie['directors'].apply(lambda x : x.split('|') if '|' in x else [x])
# this make the x to list with(using this  [x]) and  also disjoint 2 director with split('|')


In [13]:
# reset the index : 
movie.reset_index(drop=True, inplace= True)

In [14]:
movie['title'] = movie['title'].apply(lambda x : [x])

In [15]:
movie['tag'] = movie['title'] + movie['genre'] + movie['writers']+ movie['actors'] + movie['directors']

In [16]:
def jont(obj): 
    l=[]
    for i in obj: 
        l.append(i.replace(' ','').lower())
    return l 
# this function remove the spaces and convert the lower case :

In [17]:
movie['tag'] = movie['tag'].apply(jont)

In [18]:
movie['tag'] = movie['tag'].apply(lambda x : " ".join(x)) 
# this make all eliment of the x list to string by joining spaces between them

In [19]:
movie['title'] = movie['title'].apply(lambda x: str(x[0]))
# it will make the list to string : 

In [20]:
# now make the new data frame for the vectorization : 
df = movie[['imdbId','title','tag']]

#  Word Vectorization 

• vectorization is process in which we map words or phrases from vocabulary to a corresponding vector of real numbers which is used to find word predictions, similarities etc

In [21]:
from sklearn.feature_extraction.text import CountVectorizer 
cv = CountVectorizer (stop_words='english',max_features = 2000)
# this is the vetorization of the given tag colum  : 
# this will give the top 2000 most used word from the tag colum 

In [22]:
vectors = cv.fit_transform(df['tag'])

In [23]:
from sklearn.metrics.pairwise import cosine_similarity 

similarity_fector = cosine_similarity(vectors)

In [24]:
def recomendation ( movie_name ):
    movie_index = df[df['title'] == movie_name].index[0] 
    similer_movie_index = sorted(enumerate(similarity_fector[movie_index]),reverse=True, key = lambda x : x[1])[1:6]
    movie_list = [] 
    for i in similer_movie_index : 
        movie_list.append(df.iloc[i[0],1]) 
    return movie_list[0],movie_list[1],movie_list[2],movie_list[3],movie_list[4]

 # hear in this function i[1] is use to fetch the every 2 eliment of the tuple of enumarate of the similarity_fector 
   # and iloc[x , 1] (1 is use for fetch the title name ) 

In [25]:
# make prediction : 
recomendation('Saawariya') 

('Devdas',
 'Chori Chori Chupke Chupke',
 'Goliyon Ki Rasleela Ram-Leela',
 'Yeh Kaisi Mohabbat',
 'Black')

In [26]:
# dump  necessary information using pickle : 

import pickle  
pickle.dump(recomendation ,open('/Users/user/Desktop/ashish/projects/recommendor.pkl','wb')) 
pickle.dump(df,open('/Users/user/Desktop/ashish/projects/df.pkl','wb'))
pickle.dump(similarity_fector,open('/Users/user/Desktop/ashish/projects/sf.pkl','wb'))